Aufgaben Linear Networks
========================



## Imports



the common imports:



In [54]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import numpy as np

## Linear Regression Network



In the following exercise you should create a linear regression
model from scratch and test it on some synthetically created data:



In [55]:
def synthetic_data(w, b, num_examples):  #@save
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    # Rauschen dazu addieren, Form von y
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2

n_samples=100
X, y = synthetic_data(true_w, true_b, n_samples)
K = 2
print(X.shape, y.shape)

torch.Size([100, 2]) torch.Size([100, 1])


Our goal is to fit a simple regression model with Batch Gradient Descent.
We start with randomly chosen values for the weights and zero bias.
First, implement the function below.



In [56]:
torch.manual_seed(0)
w = torch.normal(0, 0.01, size=(K, 1))
b = torch.zeros(1)
print(X.shape)
print(w.shape)
print(b.shape)

def linreg(X, w, b):
    """The linear regression model."""
    return torch.matmul(X, w) + b

torch.Size([100, 2])
torch.Size([2, 1])
torch.Size([1])


Now we need to define the loss functions to be used:



In [57]:
def squared_loss(y_hat, y):
    """ Compute the sum of the quadratic errors """
    assert(len(y_hat) == len(y))
    squares = torch.zeros(0, 0)
    squares.to(torch.float64)
    for i in range(0, len(y_hat)):
        squares = (y_hat[i] - y[i])**2
    return squares

Now we need to implement the training loop for Gradient Descent.
You should not use `autograd` for computing the gradient, instead
build on the closed formula presented in the lecture.



In [58]:
step = 0.1
n_epoch = 100

loss_arr = np.zeros(n_epoch) # to record current loss
print(f"True weights: {true_w}")

for epoch in range(n_epoch):
    # 1.  Compute the prediction y_hat
    y_hat = linreg(X, w, b)
    # remember the loss for plotting it later
    loss_arr[epoch] = squared_loss(y_hat, y)
    # 2. Use y_hat and y to compute the gradients
    gradient_vals = torch.zeros(len(X), len(w))
    #bias ergänzen!!!!!!!!!!!!
    for i in range(0, len(X)):
        gradient_vals[i] = (y_hat[i] - y[i]) * X[i]
    gradients = torch.mean(gradient_vals, 0)
    # 3. Update the parameters
    w = w - step * gradients.reshape(-1, 1)
    if epoch % 10 == 0:
        print(f"Epoch({epoch}):")
        print(f"  - Gradients: {gradients}")
        print(f"  - New Weights: {w}")
        print(f"  - New Bias: {b}")
print(f"Final weights: {w}")
print(f"Final bias: {b}")
plt.plot(loss_arr)

True weights: tensor([ 2.0000, -3.4000])
Epoch(0):
  - Gradients: tensor([-1.3542,  3.0331])
  - New Weights: tensor([[ 0.1508],
        [-0.3062]])
  - New Bias: tensor([0.])
Epoch(10):
  - Gradients: tensor([-0.7343,  1.1132])
  - New Weights: tensor([[ 1.1425],
        [-2.1220]])
  - New Bias: tensor([0.])
Epoch(20):
  - Gradients: tensor([-0.3669,  0.4226])
  - New Weights: tensor([[ 1.6564],
        [-2.7991]])
  - New Bias: tensor([0.])
Epoch(30):
  - Gradients: tensor([-0.1754,  0.1656])
  - New Weights: tensor([[ 1.9071],
        [-3.0601]])
  - New Bias: tensor([0.])
Epoch(40):
  - Gradients: tensor([-0.0817,  0.0668])
  - New Weights: tensor([[ 2.0253],
        [-3.1638]])
  - New Bias: tensor([0.])
Epoch(50):
  - Gradients: tensor([-0.0375,  0.0276])
  - New Weights: tensor([[ 2.0799],
        [-3.2061]])
  - New Bias: tensor([0.])
Epoch(60):
  - Gradients: tensor([-0.0170,  0.0116])
  - New Weights: tensor([[ 2.1048],
        [-3.2237]])
  - New Bias: tensor([0.])
Epoch(70

## Linear networks with autograd



The goal now is to use `autograd` the compute the gradient.
You can use the same skeleton as before



In [59]:
torch.manual_seed(0)
w = torch.normal(0, 0.01, size=(K,1), requires_grad=True)
print(w)
b = torch.zeros(1)

step = 0.1
n_epoch = 100

loss_arr = torch.zeros(n_epoch) # to record current loss

for epoch in range(n_epoch):
    # 1.  Compute the prediction y_hat
    y_hat = linreg(X, w, b)
    # remember the loss for plotting it later
    loss = squared_loss(y_hat, y)
    loss_arr[epoch] = loss.detach()
    loss.backward()
    # 2. Use the computed loss to compute the gradients
    gradients = torch.zeros(2)
    for i in range(0, len(w)):
        gradients[i] = w.grad[i]
        w.grad[i].zero_()
    # 3. Update the parameters, remember to zero the gradients
    w = w - step * gradients.reshape(-1, 1)
    
    if epoch % 10 == 0:
        print(f"Epoch({epoch}):")
        print(f"  - Gradients: {gradients}")
        print(f"  - New Weights: {w}")

plt.plot(loss_arr)

tensor([[ 0.0154],
        [-0.0029]], requires_grad=True)
Epoch(0):
  - Gradients: tensor([0.6448, 7.8065])
  - New Weights: tensor([[-0.0491],
        [-0.7836]], grad_fn=<SubBackward0>)


C:\Users\steffen.lang\Anaconda3\lib\site-packages\torch\_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:482.)
  return self._grad


TypeError: 'NoneType' object is not subscriptable

## Linear classification



We  want to implement a linear network for classification.
We use the famous IRIS data set as an example.



In [60]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

This time the network is implemented as a class,
the only thing missing is the implementation of the softmax function,
for example
$$
softmax(y)_1 = \frac{e^{y_1}}{ \sum_{i=1}^p  e^{y_i} }.
$$
You have to implement it below.



In [61]:
def softmax(y):
    y_exp = torch.exp(y)
    partition = y_exp.sum(axis=1, keepdims=True)
    return y_exp / partition

print(softmax(torch.normal(0, 0.01, size=(3, 1), dtype=torch.float64)))

class SoftmaxNetwork:

    def __init__(self, num_input, num_output, dtype=torch.float64):
        """
        Args:
            num_input: dimension of input space
            num_output: number if output classes
        """
        self.w = torch.randn((num_input,num_output),
                             dtype=dtype).requires_grad_(True)
        self.b = torch.randn(num_output, dtype=dtype).requires_grad_(True)

    def forward(self, X):
        """
        Args:
            X: tensor of shape (n, d)
        """
        y = (X @ self.w + self.b)
        return softmax(y)

tensor([[1.],
        [1.],
        [1.]], dtype=torch.float64)


Next we have to implement the cross entropy loss, it is already finished:



In [62]:
def cross_entropy(y_hat, y):
    return (-torch.log(y_hat[range(len(y_hat)), y])).mean()

Note that this implementation does not require a one-hot-encoding for $y$
(but there is one side effect: $y$ has to be of type `torch.int64`!).

The final step is to implement a function that runs the training for us:



In [63]:
def run_training(net, X, y, f_loss, n_epochs, lr=0.1):
    """ Run the training.
    Args:
       net: an instance of SoftmaxNetwork
       X, y: training data
       f_loss: the loss function
       n_epochs: number of epochs
       lr: the learning rate
    
    Returns:
      training loss: np.array (loss per epoch)
    """
    for epoch in range(0, n_epochs):
        y_hat = net.forward(X)
        loss = f_loss(y_hat, y)
        # no idea how to do shit with this autograd 
    return loss

Now train the model, don't forget to cast X and y to Pytorch tensors.



In [64]:
# TODO: cast X and y
X = torch.from_numpy(X)
print(X.shape)
y = torch.reshape(torch.from_numpy(y), (-1, 1))
print(y.shape)
print(y)

net = SoftmaxNetwork(4,3)
train_loss = run_training(net, X, y,  cross_entropy, n_epochs=100, lr=0.2)

plt.plot(train_loss)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.title('Learning curve');

torch.Size([150, 4])
torch.Size([150, 1])
tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [

IndexError: tensors used as indices must be long, byte or bool tensors

1.  Run the training several times, and observe the different learning curves.
2.  Try the same with a lower learning rate, say $lr=0.05$. Do you see any differences?

Finally check the accuracy of the model, that is the fraction of correctly predicted examples.
Of course this is on training only. If you like you can try to split the
data into train and test and evaluate your network on the test data set.
A useful function for this is `train_test_split` found in `sklearn.model_selection`.

